In [1]:
#!pip install selenium kafka-python

In [2]:
# producer.py
from selenium import webdriver
from kafka import KafkaProducer
from selenium.webdriver.common.by import By
import json
import time
import pandas as pd

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run without GUI on VM
driver = webdriver.Chrome(options=options)

In [4]:
domains = ["https://watch.cricstream.me/t20-streams", "https://qatarstreams.me/schedule/sports-stream", "https://mainstreams.io/cricketstreams/live", "https://daddylivehd1.click"]
event_name = "Sunrisers Hyderabad vs Gujarat Titans"

In [5]:
producer = KafkaProducer(bootstrap_servers='20.83.172.165:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [6]:

for domain in domains:
    try:
        driver.get(domain)
        time.sleep(2)
        # Updated to use find_elements with By.TAG_NAME
        links = [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, 'a') 
                 if event_name.lower() in (a.get_attribute('href') or '').lower() or event_name.lower() in (a.text or '').lower()]
        for link in links:
            link_data = {"domain": domain, "link": link, "timestamp": time.strftime("%Y-%m-%dT%H:%M:%S")}
            producer.send('piracy-links', value=link_data)

            driver.get(link)
            time.sleep(2)
            screenshot = f"screenshot_{link.split('/')[-1]}.png"
            driver.save_screenshot(screenshot)
            iframe = driver.find_element(By.TAG_NAME, 'iframe').get_attribute('src') if driver.find_elements(By.TAG_NAME, 'iframe') else "N/A"
            evidence_data = {"link": link, "screenshot": screenshot, "iframe": iframe}
            producer.send('piracy-evidence', value=evidence_data)
    except Exception as e:
        print(f"Error accessing {domain}: {e}")

In [7]:
driver.quit()
producer.flush()